In [6]:
# I chose the Exponential Smoothing (Holt-Winters) model

# assignment1.py - Complete and Correct Solution

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Read data from the provided URLs
TRAIN_URL = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv"
TEST_URL = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_test.csv"

# Load and preprocess the training data
train_df = pd.read_csv(TRAIN_URL)
train_df['Timestamp'] = pd.to_datetime(train_df['Timestamp'])
train_df = train_df.set_index('Timestamp').sort_index().asfreq('H')

# Extract and prepare the target variable
y_train = train_df['trips'].copy()
y_train = pd.to_numeric(y_train, errors='coerce')

# Handle missing values - forward fill then backward fill
y_train = y_train.fillna(method='ffill').fillna(method='bfill')

# Create the Exponential Smoothing model (unfitted instance)
model = ExponentialSmoothing(
    y_train,
    trend='add',
    seasonal='add',
    seasonal_periods=168,  # Weekly seasonality (24*7=168 hours)
    initialization_method='estimated'
)

# Fit the model to create modelFit
modelFit = model.fit(
    optimized=True,
    use_brute=True,
    remove_bias=True
)

# Load the test data
test_df = pd.read_csv(TEST_URL)
test_df['Timestamp'] = pd.to_datetime(test_df['Timestamp'])

# Generate forecasts for January 2019 (744 hours)
forecast = modelFit.forecast(steps=744)

# Convert to the exact format required by the autograder
pred = np.array(forecast, dtype=np.float64).flatten()

# Ensure no negative predictions
pred = np.clip(pred, 0, None)

# Verify the output format meets requirements
assert len(pred) == 744, f"Prediction length should be 744, got {len(pred)}"
assert isinstance(pred, np.ndarray), "Prediction should be a numpy array"
assert pred.ndim == 1, "Prediction should be 1-dimensional"

/tmp/ipython-input-4203514758.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train_df = train_df.set_index('Timestamp').sort_index().asfreq('H')
/tmp/ipython-input-4203514758.py:23: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y_train = y_train.fillna(method='ffill').fillna(method='bfill')
